## ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

['/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-04-events.csv', '/workspace/home/event_data/2018-11-08-events.csv', '/workspace/home/event_data/2018-11-05-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-14-events.csv', '/workspace/home/event_data/2018-11-16-events.csv', '/workspace/home/event_data/2018-11-18-events.csv', '/workspace/home/event_data/2018-11-22-events.csv', '/workspace/home/event_data/2018-11-09-events.csv', '/workspace/home/event_data/2018-11-26-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-19-events.csv', '/workspace/home/event_data/2018-11-01-events.csv', '/workspace/home/event_data/2018-11-27-events.csv', '/workspace/home/event_data/2018-11-03-events.csv', '/workspace/home/event_data/2018-11-24-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-10-events.csv', '/workspace

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line)


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Writing Apache Cassandra code

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e) 

#### Set Keyspace

In [7]:
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

## Queries to ask the following three questions on the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1 - Question and Solution

#### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session_and_item "
query = query + "(session_id int, item_in_session int, artist text, length float, song text, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_session_and_item (session_id, item_in_session, artist, length, song)"
        query = query + "VALUES (%s, %s,%s, %s,%s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9] ))
        except Exception as e:
            print(e)
            print([int(line[8]), int(line[3]), line[0], float(line[5]), line[9]])
            raise (e)


######  Solution query 1: The primary key is made of of the (session_id, item_in_session) with session_id as the partition key and item_in_session for clustering

In [10]:
query = 'SELECT artist, song, length FROM songs_by_session_and_item WHERE session_id = 338 and item_in_session = 4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print ("Artist".ljust(9), "Song".ljust(31), "Length".ljust(20), sep=" | ")
print ("".ljust(9, "-"), "".ljust(31, "-"), "".ljust(20, "-"), sep=" | ")
for row in rows:
    print (row.artist, row.song, row.length, sep=" | ")

Artist    | Song                            | Length              
--------- | ------------------------------- | --------------------
Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


### Query 2 - Question and Solution

#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10 , sessionid = 182

In [11]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user_and_session "
query += "(user_id int, session_id int, item_in_session int, artist text, first_name text, last_name text, song text, PRIMARY KEY ((user_id, session_id), item_in_session) )"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [12]:

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_and_session (user_id, session_id, item_in_session, artist, first_name, last_name, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s,%s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9] ))
        except Exception as e:
            print(e)
            print([int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]])
            raise (e)


######  Solution query 2: The composite primary key is made of of the ((user_id, session_id), item_in_session) with user_id/session_id as the composite key and item_in_session for clustering

In [13]:
query = 'SELECT artist, song, first_name, last_name FROM songs_by_user_and_session WHERE user_id = 10 and session_id = 182'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print ("Artist".ljust(17), "Song".ljust(52), "First Name".ljust(12), "Last Name".ljust(12), sep="|")
print ("".ljust(17, "-"), "".ljust(52, "-"), "".ljust(12, "-"), "".ljust(12, "-"), sep="|")
for row in rows:
    print (row.artist.ljust(17), row.song.ljust(52), row.first_name.ljust(12), row.last_name.ljust(12), sep="|")

Artist           |Song                                                |First Name  |Last Name   
-----------------|----------------------------------------------------|------------|------------
Down To The Bone |Keep On Keepin' On                                  |Sylvie      |Cruz        
Three Drives     |Greece 2000                                         |Sylvie      |Cruz        
Sebastien Tellier|Kilometer                                           |Sylvie      |Cruz        
Lonnie Gordon    |Catch You Baby (Steve Pitron & Max Sanna Radio Edit)|Sylvie      |Cruz        


### Query 3 - Question and Solution

#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

In [14]:

query = "CREATE TABLE IF NOT EXISTS songs_by_title_and_session "
query += "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id) )"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_title_and_session (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4] ))
        except Exception as e:
            print(e)
            print([line[9], int(line[10]), line[1], line[4]])
            raise (e)


######  Solution query 3: The composite primary key is made of of the (song, user_id) with song, session_id as the partition key and session_id for clustering

In [16]:
query = "SELECT first_name, last_name FROM songs_by_title_and_session WHERE song = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print ("First Name".ljust(20), "Last Name".ljust(20), sep=" | ")
print ("".ljust(21, "-"), "".ljust(20, "-"), sep="|")
for row in rows:
    print (row.first_name.ljust(20), row.last_name.ljust(20), sep=" | ")

First Name           | Last Name           
---------------------|--------------------
Jacqueline           | Lynch               
Tegan                | Levine              
Sara                 | Johnson             


### Drop the tables before closing out the sessions

In [17]:
query1 = "DROP TABLE songs_by_session_and_item"
query2 = "DROP TABLE songs_by_user_and_session"
query3 = "DROP TABLE songs_by_title_and_session"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()